# Used for predicting ALSFRS_slope
see https://www.synapse.org/#!Synapse:syn2873386/wiki/ .
We assumed data is vectorized + clustered + 6 features were selected

In [1]:
from IPython.display import display

import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model
from vectorizing_funcs import *
from modeling_funcs import *

## Revectorize the selected data
We now reload the metadata and the 6 attributes selected per cluster

In [9]:
# load all metadata
all_feature_metadata = pickle.load( open('../all_feature_metadata.pickle', 'rb') )
train_data_means = pickle.load( open('../all_data_means.pickle', 'rb') )
train_data_std = pickle.load( open('../all_data_std.pickle', 'rb') )
best_features_per_cluster = pickle.load( open('../best_features_per_cluster.pickle', 'rb') )

# reload the data imputed only for the 6 selected attributes per cluster
df = pd.read_csv('../all_data_selected.csv', sep='|', index_col=False, dtype="unicode")
vectorized, _ = vectorize(df, all_feature_metadata)
vectorized.head()


,Bulbar,Limb,Limb and Bulbar,pulse_last,pulse_mean_slope,pulse_mean,pulse_pct_diff,White Blood Cell (WBC)_last,White Blood Cell (WBC)_mean_slope,White Blood Cell (WBC)_mean,...,Platelets_mean,Platelets_pct_diff,Blood Urea Nitrogen (BUN)_last,Blood Urea Nitrogen (BUN)_mean_slope,Blood Urea Nitrogen (BUN)_mean,Blood Urea Nitrogen (BUN)_pct_diff,ALT(SGPT)_last,ALT(SGPT)_mean_slope,ALT(SGPT)_mean,ALT(SGPT)_pct_diff
SubjectID,,,,,,,,,,,,,,,,,,,,,
3551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,56,0.148810,41.666667,0.009524
4752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,33,0.195312,30.000000,0.005859
7399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,21,0.179731,23.200000,0.000549
8471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,32,0.031593,25.666667,0.004300
10244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,20.000000,NaN


In [15]:
vectorized.index = vectorized.index.astype(str)
d = vectorized.describe().T
d[d['count'] > 0.0].sort("count", ascending=False).head()


,count,mean,std,min,25%,50%,75%,max
Creatinine_mean,1311,69.035754,17.447380,19.250000,56.875000,68.000000,80.000000,150.280000
Creatinine_last,1311,68.065834,18.581242,17.680000,53.040000,67.000000,79.560000,159.120000
Creatinine_mean_slope,1153,-0.208227,1.035913,-10.253129,-0.250467,-0.021667,0.097143,4.544464
Creatinine_pct_diff,1153,0.045020,1.542236,-0.015385,-0.001832,0.000000,0.000444,52.367476
respiratory_last,881,3.743473,0.509038,1.000000,4.000000,4.000000,4.000000,4.000000


In [16]:
slope = pd.read_csv('../all_slope.csv', sep = '|', index_col="SubjectID")
slope.index = slope.index.astype(str)

# if we have a subject missing all selected features, fill him with missing values right before normalizing
vectorized = vectorized.join(slope, how = 'right')
vectorized = vectorized.drop('ALSFRS_slope', 1)
normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
print normalized.shape
normalized.head()

(2205, 140)


,family_ALS_hist_last,Bulbar,Limb,Limb and Bulbar,weight_last,weight_mean_slope,weight_mean,weight_pct_diff,Sodium_last,Sodium_mean_slope,...,Platelets_pct_diff,if_use_Riluzole_last,Protein_last,Protein_mean_slope,Protein_mean,Protein_pct_diff,ALT(SGPT)_last,ALT(SGPT)_mean_slope,ALT(SGPT)_mean,ALT(SGPT)_pct_diff
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,0,1.993896,-1.949922,-0.085475,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
649,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1234,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2492,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2956,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
d = normalized.describe().T
d[d['std'] > 0.0].sort("std", ascending=False).head()

,count,mean,std,min,25%,50%,75%,max
Creatinine_pct_diff,2205,0.007501,0.920499,-0.035523,-0.022924,-0.019591,0.000000,43.208702
ALSFRS_Total_mean_slope,2205,-0.041799,0.789588,-5.755488,0.000000,0.000000,0.000000,23.440788
ALSFRS_Total_pct_diff,2205,-0.046244,0.788882,-6.591089,0.000000,0.000000,0.000000,23.361300
Creatinine_mean,2205,-0.020279,0.781100,-2.924466,-0.278471,0.000000,0.079928,4.682601
Creatinine_last,2205,-0.020407,0.769714,-2.740953,-0.365540,0.000000,0.108254,4.856932


In [19]:
clusters = pd.read_csv('../all_forest_clusters.csv', sep = '|', index_col="SubjectID")

clusters.index = clusters.index.astype(str)
normalized.index = normalized.index.astype(str)

X = normalized.join(clusters)
Y = slope.join(clusters)

print Y.shape, X.shape, clusters.shape
print clusters.groupby('cluster').size()
display(Y.head(3))


(2205, 2) (2205, 141) (2205, 1)
cluster
0    441
1    441
2    441
3    441
4    441
dtype: int64


,ALSFRS_slope,cluster
SubjectID,,
533,-0.965608,3
649,-0.921717,1
1234,-0.914787,4


## Train a prediction model per cluster

In [20]:

model_per_cluster = get_model_per_cluster(X, Y)
    

cluster: 3 size: (441L,)
	 RMS error (0 is perfect): 0.43
	 explained variance score (1 is perfect): 0.11
3 sample predictions:  [-0.92699149 -0.49863191 -0.65804949]
cluster: 1 size: (441L,)
	 RMS error (0 is perfect): 0.52
	 explained variance score (1 is perfect): 0.16
3 sample predictions:  [-0.60784338 -0.93335658 -0.66649377]
cluster: 4 size: (441L,)
	 RMS error (0 is perfect): 0.35
	 explained variance score (1 is perfect): 0.10
3 sample predictions:  [-0.23361618 -0.29984098 -0.30987943]
cluster: 0 size: (441L,)
	 RMS error (0 is perfect): 0.59
	 explained variance score (1 is perfect): 0.15
3 sample predictions:  [-1.09530449 -1.40066062 -1.30984556]
cluster: 2 size: (441L,)
	 RMS error (0 is perfect): 0.45
	 explained variance score (1 is perfect): 0.14
3 sample predictions:  [-0.82416934 -0.56561608 -0.81840493]


C:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:444: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


In [21]:
with open("../model_per_cluster.pickle", "wb") as output_file:
    pickle.dump(model_per_cluster, output_file)


## Apply the model on both `train` and `test`

In [22]:

for t in ['all', 'test']:
    print t
    df = pd.read_csv('../' + t + '_data_selected.csv', sep='|', index_col=False, dtype="unicode")
    vectorized, _ = vectorize(df, all_feature_metadata)
    normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
    clusters = pd.read_csv('../' + t + '_forest_clusters.csv', sep = '|', index_col=0)

    clusters.index = clusters.index.astype(str)
    normalized.index = normalized.index.astype(str)

    X = normalized.join(clusters)
    print X.groupby('cluster').size()
    pred = X.apply(apply_model, args=[model_per_cluster], axis = 1)
    pred.to_csv('../' + t + '_prediction.csv',sep='|')

pred.head()

all
cluster
0    441
1    441
2    441
3    441
4    441
dtype: int64
test
cluster
0    118
1    124
2    118
3    120
4    111
dtype: int64


,cluster,prediction
SubjectID,,
750059,0,-1.172532
762989,0,-1.459922
764548,0,-0.980801
768977,0,-0.932015
770063,0,-0.462370
